[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/multi-tenancy/multi-tenancy-automatic-tenant-creation.ipynb)

In [1]:
import weaviate
client = weaviate.connect_to_embedded(
    version="1.25.2"
)
client.is_ready()

Binary /Users/dudanogueira/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.25.2/weaviate-v1.25.2-Darwin-all.zip
Started /Users/dudanogueira/.cache/weaviate-embedded: process ID 60656


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-06-02T14:11:44-03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-06-02T14:11:44-03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-06-02T14:11:44-03:00"}
{"level":"info","msg":"open cluster service","servers":{"Embedded_at_8079":60843},"time":"2024-06-02T14:11:44-03:00"}
{"address":"192.168.28.99:60844","level":"info","msg":"starting cloud rpc server ...","time":"2024-06-02T14:11:44-03:00"}
{"level":"info","msg":"starting raft sub-system ...","time":"2024-06-02T14:11:44-03:00"}
{"address":"192.168.28.99:60843","level":"info","msg":"tcp transport","tcpMa

True

{"action":"bootstrap","level":"info","msg":"node reporting ready, node has probably recovered cluster from raft config. Exiting bootstrap process","time":"2024-06-02T14:11:47-03:00"}
{"action":"telemetry_push","level":"info","msg":"telemetry started","payload":"\u0026{MachineID:f0ed3f27-d1ec-46a4-8ffd-9f8813a6e1a4 Type:INIT Version:1.25.2 NumObjects:0 OS:darwin Arch:arm64 UsedModules:[generative-ollama generative-openai text2vec-ollama text2vec-openai]}","time":"2024-06-02T14:11:49-03:00"}


In [2]:
from weaviate.classes.config import Configure

client.collections.delete("CollectionWithAutoMTEnabled")
multi_collection = client.collections.create(
    name="CollectionWithAutoMTEnabled",
    # Enable automatic tenant creation
    multi_tenancy_config=Configure.multi_tenancy(
        enabled=True, auto_tenant_creation=True
    ),
)

In [3]:
# lets import data in batch to newly created tenants

data = [
    {"tenant": "tenant1", "text": "Text from batch for tenant1"},
    {"tenant": "tenant2", "text": "Text from batch for tenant2"},
]

with client.batch.fixed_size(batch_size=200) as batch:
    for obj in data:
        properties = {
            "text": obj["text"],
        }
        batch.add_object(
            collection="CollectionWithAutoMTEnabled",
            properties=properties,
            tenant=obj["tenant"],
        )

{"level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/collectionwithautomtenabled/tenant2/proplengths does not exist, creating new tracker","time":"2024-06-02T14:12:08-03:00"}
{"level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/collectionwithautomtenabled/tenant1/proplengths does not exist, creating new tracker","time":"2024-06-02T14:12:08-03:00"}
{"level":"info","msg":"Created shard collectionwithautomtenabled_tenant2 in 1.988917ms","time":"2024-06-02T14:12:08-03:00"}
{"level":"info","msg":"Created shard collectionwithautomtenabled_tenant1 in 2.020584ms","time":"2024-06-02T14:12:08-03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-06-02T14:12:08-03:00","took":53334}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time

In [4]:
tenant1 = client.collections.get("CollectionWithAutoMTEnabled").with_tenant("tenant1")
tenant1.query.fetch_objects().objects[0].properties

{'text': 'Text from batch for tenant1'}

In [5]:
tenant1 = client.collections.get("CollectionWithAutoMTEnabled").with_tenant("tenant2")
tenant1.query.fetch_objects().objects[0].properties

{'text': 'Text from batch for tenant2'}

In [6]:
client.collections.get("CollectionWithAutoMTEnabled").tenants.get()

{'tenant1': Tenant(name='tenant1', activityStatus=<TenantActivityStatus.HOT: 'HOT'>),
 'tenant2': Tenant(name='tenant2', activityStatus=<TenantActivityStatus.HOT: 'HOT'>)}

**Note:** introduced only in 1.25.2

In [7]:
# create tenant also from single inserts
tenant3 = client.collections.get("CollectionWithAutoMTEnabled").with_tenant("tenant3")
tenant3.data.insert({"text": "This a single insert for tenant 3"})

{"level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/collectionwithautomtenabled/tenant3/proplengths does not exist, creating new tracker","time":"2024-06-02T14:13:30-03:00"}
{"level":"info","msg":"Created shard collectionwithautomtenabled_tenant3 in 1.964083ms","time":"2024-06-02T14:13:30-03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-06-02T14:13:30-03:00","took":53417}


UUID('278b9e23-9506-424c-87aa-1aebf7f633e1')

In [8]:
client.collections.get("CollectionWithAutoMTEnabled").tenants.get()

{'tenant1': Tenant(name='tenant1', activityStatus=<TenantActivityStatus.HOT: 'HOT'>),
 'tenant2': Tenant(name='tenant2', activityStatus=<TenantActivityStatus.HOT: 'HOT'>),
 'tenant3': Tenant(name='tenant3', activityStatus=<TenantActivityStatus.HOT: 'HOT'>)}